In [ ]:
!apt update
!apt install -y ocl-icd-opencl-dev pocl-opencl-icd clinfo

In [7]:
%%writefile reduction.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

const int N = 1 << 20;
const int BLOCK_SIZE = 256;

// Макрос для проверки ошибок CUDA
#define CUDA_CHECK(call)                                   \
    do {                                                   \
        cudaError_t err = call;                            \
        if (err != cudaSuccess) {                          \
            cout << "CUDA ошибка: "                        \
                 << cudaGetErrorString(err) << endl;      \
            return 1;                                      \
        }                                                  \
    } while (0)

__global__ void reductionKernel(const float* input, float* partial, int n) {
    __shared__ float sharedData[BLOCK_SIZE];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        sharedData[tid] = input[idx];
    } else {
        sharedData[tid] = 0.0f;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sharedData[tid] += sharedData[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        partial[blockIdx.x] = sharedData[0];
    }
}

int main() {
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    float cpuSum = 0.0f;
    for (int i = 0; i < N; i++) {
        cpuSum += h_data[i];
    }

    float *d_data, *d_partial;

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float)));
    CUDA_CHECK(cudaMalloc(&d_partial, gridSize * sizeof(float)));

    CUDA_CHECK(cudaMemcpy(d_data, h_data,
                           N * sizeof(float),
                           cudaMemcpyHostToDevice));

    reductionKernel<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);

    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    float* h_partial = new float[gridSize];
    CUDA_CHECK(cudaMemcpy(h_partial, d_partial,
                           gridSize * sizeof(float),
                           cudaMemcpyDeviceToHost));

    float gpuSum = 0.0f;
    for (int i = 0; i < gridSize; i++) {
        gpuSum += h_partial[i];
    }

    cout << "Сумма на CPU: " << cpuSum << endl;
    cout << "Сумма на GPU: " << gpuSum << endl;

    if (abs(cpuSum - gpuSum) < 1e-3) {
        cout << "Результат корректен" << endl;
    } else {
        cout << "Ошибка в вычислениях" << endl;
    }

    delete[] h_data;
    delete[] h_partial;
    cudaFree(d_data);
    cudaFree(d_partial);

    return 0;
}


Overwriting reduction.cu


In [9]:
!nvcc -arch=sm_75 reduction.cu -o reduction
!./reduction


Сумма на CPU: 1.04858e+06
Сумма на GPU: 1.04858e+06
Результат корректен


In [10]:
%%writefile prefix_sum.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Размер массива (один блок)
const int N = 256;

// CUDA-ядро префиксной суммы с использованием shared memory
__global__ void prefixSumKernel(const float* input, float* output) {
    __shared__ float temp[N];

    int tid = threadIdx.x;

    // Загрузка данных в shared memory
    temp[tid] = input[tid];
    __syncthreads();

    // Алгоритм префиксной суммы (Hillis-Steele)
    for (int offset = 1; offset < N; offset <<= 1) {
        float value = 0.0f;
        if (tid >= offset) {
            value = temp[tid - offset];
        }
        __syncthreads();
        temp[tid] += value;
        __syncthreads();
    }

    // Запись результата в глобальную память
    output[tid] = temp[tid];
}

int main() {
    float h_input[N];
    float h_output[N];
    float h_cpu[N];

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_input[i] = 1.0f;
    }

    // Последовательная префиксная сумма на CPU
    h_cpu[0] = h_input[0];
    for (int i = 1; i < N; i++) {
        h_cpu[i] = h_cpu[i - 1] + h_input[i];
    }

    float *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(float));
    cudaMalloc(&d_output, N * sizeof(float));

    cudaMemcpy(d_input, h_input, N * sizeof(float), cudaMemcpyHostToDevice);

    // Запуск ядра (один блок)
    prefixSumKernel<<<1, N>>>(d_input, d_output);
    cudaDeviceSynchronize();

    cudaMemcpy(h_output, d_output, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Проверка корректности
    bool correct = true;
    for (int i = 0; i < N; i++) {
        if (abs(h_output[i] - h_cpu[i]) > 1e-5) {
            correct = false;
            break;
        }
    }

    cout << "Первые 10 элементов результата:\n";
    for (int i = 0; i < 10; i++) {
        cout << h_output[i] << " ";
    }
    cout << endl;

    if (correct) {
        cout << "Результат корректен" << endl;
    } else {
        cout << "Ошибка в вычислениях" << endl;
    }

    cudaFree(d_input);
    cudaFree(d_output);

    return 0;
}


Writing prefix_sum.cu


In [11]:
!nvcc -arch=sm_75 prefix_sum.cu -o prefix_sum
!./prefix_sum


Первые 10 элементов результата:
1 2 3 4 5 6 7 8 9 10 
Результат корректен


In [12]:
%%writefile performance_analysis.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

const int BLOCK_SIZE = 256;

__global__ void reductionKernel(const float* input, float* partial, int n) {
    __shared__ float data[BLOCK_SIZE];

    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + tid;

    data[tid] = (idx < n) ? input[idx] : 0.0f;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            data[tid] += data[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        partial[blockIdx.x] = data[0];
    }
}

void cpuReduction(const float* data, int n) {
    volatile float sum = 0.0f;
    for (int i = 0; i < n; i++) {
        sum += data[i];
    }
}

int main() {
    int sizes[] = {1024, 16384, 262144, 1048576};

    for (int s = 0; s < 4; s++) {
        int N = sizes[s];
        cout << "\nРазмер массива: " << N << endl;

        float* h_data = new float[N];
        for (int i = 0; i < N; i++) {
            h_data[i] = 1.0f;
        }

        float *d_data, *d_partial;
        int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

        cudaMalloc(&d_data, N * sizeof(float));
        cudaMalloc(&d_partial, gridSize * sizeof(float));
        cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

        // GPU тайминг
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        reductionKernel<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float gpuTime;
        cudaEventElapsedTime(&gpuTime, start, stop);

        // CPU тайминг
        auto cpuStart = high_resolution_clock::now();
        cpuReduction(h_data, N);
        auto cpuEnd = high_resolution_clock::now();
        double cpuTime = duration<double, milli>(cpuEnd - cpuStart).count();

        cout << "CPU редукция: " << cpuTime << " мс" << endl;
        cout << "GPU редукция: " << gpuTime << " мс" << endl;

        delete[] h_data;
        cudaFree(d_data);
        cudaFree(d_partial);
    }

    return 0;
}


Writing performance_analysis.cu


In [13]:
!nvcc -arch=sm_75 performance_analysis.cu -o perf
!./perf



Размер массива: 1024
CPU редукция: 0.003718 мс
GPU редукция: 0.12304 мс

Размер массива: 16384
CPU редукция: 0.054798 мс
GPU редукция: 0.012288 мс

Размер массива: 262144
CPU редукция: 0.85823 мс
GPU редукция: 0.033152 мс

Размер массива: 1048576
CPU редукция: 3.60545 мс
GPU редукция: 0.10736 мс
